<a href="https://colab.research.google.com/github/sazio/Transients/blob/master/Python/WLC_SSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [3]:
# Simulation's Parameters 
N_s = 588
N_p = 10 
alpha = 1
beta = 2.5
V_1 = 0.9
eps = 0.01
sigma = 0.0001
tau = 480